In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-238588
Azure region: eastus2
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-238588


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_modes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Found existing cluster

Running


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                script='train_wv.py',
                compute_target='Udacity',
                environment=sklearn_env)

#primary_metric = 'Accuracy'
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=100,
                max_concurrent_runs=4)

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_f7ca2171-e09d-40a9-863b-9f81ca8aa1b7
Web View: https://ml.azure.com/runs/HD_f7ca2171-e09d-40a9-863b-9f81ca8aa1b7?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-238588/workspaces/quick-starts-ws-238588&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-07-12T13:24:57.617029][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-07-12T13:24:58.1288391Z][SCHEDULER][INFO]Scheduling job, id='HD_f7ca2171-e09d-40a9-863b-9f81ca8aa1b7_0' 
[2023-07-12T13:24:58.1861728Z][SCHEDULER][INFO]Scheduling job, id='HD_f7ca2171-e09d-40a9-863b-9f81ca8aa1b7_1' 
[2023-07-12T13:24:58.3795193Z][SCHEDULER][INFO]Scheduling job, id='HD_f7ca2171-e09d-40a9-863b-9f81ca8aa1b7_3' 
[2023-07-12T13:24:58.2678674Z][SCHEDULER][INFO]Scheduling job, id='HD_f7ca2171-e09d-40a9-863b-9f81ca8aa1b7_2' 
[2023-07-12T13:24:58.325443][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_f7ca2171-e09d-40a9-863b-9f81ca8aa1b7',
 'target': 'Udacity',
 'status': 'Completed',
 'startTimeUtc': '2023-07-12T13:24:56.918719Z',
 'endTimeUtc': '2023-07-12T13:30:59.106997Z',
 'services': {},
 'error': {'error': {'code': 'System',
   'message': 'AzureMLCompute job failed\nFailureExitCode: The task exited with an exit code representing a failure\n\tMessage: The task process exited with an unexpected exit code\n\tAdditionalErrorCode: FailureExitCode System errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0f965154-d512-4e7b-8784-195cf5a3b4ec',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdr

In [10]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='outputs/model.joblib')

Run(Experiment: udacity-project,
Id: HD_f7ca2171-e09d-40a9-863b-9f81ca8aa1b7_3,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-238588', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-238588'), name=best_hd_model.pkl, id=best_hd_model.pkl:1, version=1, tags={}, properties={})

In [11]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.1', '--max_iter', '500']


In [ ]:
pip install azureml-dataset-runtime --upgrade

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [17]:
!pip install scikit_learn

In [21]:

from train_wv import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

SyntaxError: invalid syntax (train_wv.py, line 1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [ ]:
# Submit your automl run

experiment = Experiment(ws, 'automl')
run = experiment.submit(config=automl_config, show_out=True)

In [ ]:
# Retrieve and save your best automl model.

best_run, model = run.get_output()
best_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_run)